In [1]:
# @hidden_cell
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
CLIENT_ID = 'GQHLGSG1LUV0ZDG1M0CTN3KZV4FEHAYAGQM11NRZP2JO1MBB' # your Foursquare ID
CLIENT_SECRET = 'NH3TZNODNWWIFLTBLEVO4I2QEY5PJU2IL5V4YH5GL1GVPITA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Libraries imported.
Your credentails:
CLIENT_ID: GQHLGSG1LUV0ZDG1M0CTN3KZV4FEHAYAGQM11NRZP2JO1MBB
CLIENT_SECRET:NH3TZNODNWWIFLTBLEVO4I2QEY5PJU2IL5V4YH5GL1GVPITA


In [2]:
#within this block, we find the geographic location of Singapore
sgaddress = 'Singapore, SG'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(sgaddress)
latitude = location.latitude
longitude = location.longitude 
print(latitude, longitude)

1.3408528 103.878446863736


In [3]:
#we then in this block find restaurants within a specified radius away from the 'Singapore' geographic location
search_query = 'Restaurant'
radius = 1000

#this function returns the category of the entry.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#here we call the foursquare api and find the nearby restaurants, save as a variable.
#Then within the if statement, if there exist entries, we then write these entries to dataframe and filter for locational columns only.     
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
if results['response']['venues']!= []:
    venues = results['response']['venues']
    df = json_normalize(venues)
    filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = df.loc[:, filtered_columns]
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]   

In [4]:
#we find however there to only be upto 50 entries, so we need to make multiple calls to the api to gather data on a larger sample of singapore restaurants.
#within the following loops, we position the geographic centre of our search differently with each iteration. If there exists entries, we add these entries to the larger dataframe
#Again we filter for locational columns only, in a similar way to the previous columns. 
latitude = 1.2385
while latitude < 1.469:
    longitude = 103.614
    while longitude < 104.09:
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
        if results['response']['venues']!= []:
            df_new = json_normalize(results['response']['venues'])
            filtered_columns = ['name', 'categories'] + [col for col in df_new.columns if col.startswith('location.')] + ['id']
            df_filtered_new = df_new.loc[:, filtered_columns]
            df_filtered_new['categories'] = df_filtered_new.apply(get_category_type, axis=1)
            df_filtered_new.columns = [column.split('.')[-1] for column in df_filtered_new.columns]      
            dataframe_filtered = dataframe_filtered.append(df_filtered_new, ignore_index=True)
        longitude+=0.005 
    print(latitude) # the latitude is printed the algorithm is running correctly
    latitude+=0.01
print('Done')

#below we eradicate duplicate entries and entries from outside of Singapore.
dataframe_filtered.drop_duplicates(subset='id', inplace=True)
dataframe_filtered = dataframe_filtered[dataframe_filtered.cc == 'SG']

#here we drop entries where the category is not known then reset index
dataframe_filtered.dropna(inplace=True, subset =['categories'])
dataframe_filtered.reset_index(inplace=True, drop=True)

#here we leave only the rows of use then write the df to a csv file.
dataframe_filtered = dataframe_filtered[['name','categories','id','lat','lng']]
dataframe_filtered.to_csv('singapore_restaurant.csv')

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1.2385
1.2485
1.2585
1.2685
1.2785
1.2885
1.2985
1.3085
1.3185
1.3285
1.3385
1.3485
1.3585
1.3685
1.3785
1.3885
1.3985
1.4085
1.4185
1.4285
1.4385000000000001
1.4485000000000001
1.4585000000000001
1.4685000000000001
Done
